In [1]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

from llm_data_quality_assistant import pipeline
from llm_data_quality_assistant.corruptor import RowCorruptionTypes, CellCorruptionTypes
from llm_data_quality_assistant.enums import Models
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
corrupt_dataset = pd.read_csv("../datasets/parker_datasets/SHORT_allergen.csv")

gold_standard = pd.read_csv("../datasets/parker_datasets/gold_standard_alergene_pivoted.csv")

shortened_corrupt_df = corrupt_dataset[corrupt_dataset["code"].isin(gold_standard["code"])]

print("Shape shortened corrupt dataset:")
print(shortened_corrupt_df.shape)

p = pipeline.Pipeline(gold_standard)

merged_df = p.clean_single_dataset(shortened_corrupt_df)

shortened_merged_df = merged_df[merged_df["code"].isin(gold_standard["code"])]

print("Shape merged dataset:")
print(merged_df.shape)

print("Shape shortened merged dataset:")
print(shortened_merged_df.shape)


Shape shortened corrupt dataset:
(18, 22)
[
  {
    "code": 4104420006065,
    "nuts": 0,
    "almondnuts": 0,
    "brazil_nuts": 0,
    "macadamia_nuts": 0,
    "hazelnut": 0,
    "pistachio": 0,
    "walnut": 0,
    "cashew": 0,
    "celery": 0,
    "crustaceans": 0,
    "eggs": 1,
    "fish": 0,
    "gluten": 2,
    "lupin": 0,
    "milk": 0,
    "molluscs": 0,
    "mustard": 0,
    "peanut": 0,
    "sesame": 0,
    "soy": 1,
    "sulfite": 0
  },
  {
    "code": 4104420007963,
    "nuts": 2,
    "almondnuts": 0,
    "brazil_nuts": 0,
    "macadamia_nuts": 0,
    "hazelnut": 0,
    "pistachio": 0,
    "walnut": 0,
    "cashew": 0,
    "celery": 0,
    "crustaceans": 0,
    "eggs": 0,
    "fish": 0,
    "gluten": 0,
    "lupin": 0,
    "milk": 0,
    "molluscs": 0,
    "mustard": 0,
    "peanut": 0,
    "sesame": 0,
    "soy": 0,
    "sulfite": 0
  },
  {
    "code": 4104420007987,
    "nuts": 2,
    "almondnuts": 0,
    "brazil_nuts": 0,
    "macadamia_nuts": 0,
    "hazelnut": 0,
 